In [1]:
# !pip install pysimplegui
#pip install --force-reinstall -v "pyabf==2.3.8" #delite and reinstall the certain version
#pip install -Iv MySQL_python==1.2.2 # install the certain version
#!pip install matplotx==0.0.0

#import abf_explorer
#!pip install --upgrade numpy
import pyabf

import numpy as np

import matplotlib.pyplot as plt
#import matplotx

import statistics

from numpy.polynomial import Polynomial

import PySimpleGUI as sg

import pandas as pd

import os

In [2]:
#my_file = pyabf.ABF("2023_08_03_0054.abf")

In [31]:
def TimePeriodBorders (my_file_abf):
    sample_rate = int(my_file_abf.dataRate)
    if sample_rate == 20000: 
        # Time for PreStim and Stim for Channel 0 
        my_start10 = 0 
        my_end10 = 2800    
        my_start11 = 4000 
        my_end11 = 12800 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 28000 
        my_end20 = 30800 
        my_start21 = 32000 
        my_end21 = 40800 
        # Rin calculation
        my_Rtime_ch0 = 8000 
        my_Rtime_ch2 = 36000 
    elif sample_rate == 10000:
        # Time for PreStim and Stim for Channel 0
        my_start10 = 0 
        my_end10 = 1400     
        my_start11 = 2000 
        my_end11 = 6400 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 14000
        my_end20 = 15400
        my_start21 = 16000
        my_end21 = 20000 
        # Rin calculation
        my_Rtime_ch0 = 4000
        my_Rtime_ch2 = 18000 
    else:
        print("Check the Sampling rate!")
        return ([],[],[],[],[],[],[],[])
        
    return (my_start10, my_end10, 
            my_start11, my_end11, 
            my_start20, my_start21, 
            my_start21, my_end21, 
            my_Rtime_ch0, my_Rtime_ch2)

def Rin_R2_calculation (currents, voltages):
    my_approximation = np.polyfit (currents, voltages, 1, full = True)
    try:
        my_SEE = my_approximation[1][0]
        my_diff = voltages - statistics.mean(voltages)
        square_diff = my_diff ** 2
        my_SST = square_diff.sum()
        my_R2 = np.round((1 - my_SEE/my_SST), 3)
        my_Rin = np.round(my_approximation[0][0], 3)
    except IndexError:
        my_R2 = 'Not enought point'
        my_Rin = np.round(my_approximation[0][0], 3)
    return my_Rin, my_R2



def IVpoints (my_file_abf, 
              voltage_channel, current_channel, 
              voltage_start, voltage_end, current_startend,
              voltages_list, currents_list):
        for my_sweepNumber in range(int(my_file_abf.sweepList[-1])):
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = voltage_channel)
            if statistics.mean(my_file_abf.sweepY[voltage_start:voltage_end]) > -90 and statistics.mean(my_file_abf.sweepY[voltage_start:voltage_end]) < -50: 
                voltages_list.append(statistics.mean(my_file_abf.sweepY[voltage_start:voltage_end])) 
                my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = current_channel) 
                currents_list.append(my_file_abf.sweepY[current_startend])
            
        return currents_list, voltages_list

In [27]:
# GUI to choose folder
tres = 0
folder_path = False
while True:
    event, values = sg.Window('Choose your folder', [[sg.Text('Folder')], [sg.Input(), sg.FolderBrowse()], [sg.OK(), sg.Cancel()] ]).read(close=True)
    if event == 'Cancel':
        break
    try:
        folder_path = values[0]
        break
    except:
        print('Incorrect input')

In [32]:
file_names_list = []

Rin_ch0_list = []
R2_ch0_list = []
number_ch0_list = []

Rin_ch2_list = []
R2_ch2_list = []
number_ch2_list = []

#folder_path = 'C:/Users/user/Documents/IMBIT/05.10.2023'
file_list = os.listdir(folder_path)
#print(file_list) # List of all file names

for filename in file_list:
    print(filename)
    file_names_list.append(filename)
    file_path = os.path.join(folder_path, filename)
    my_file = pyabf.ABF(file_path)
    start10, end10, start11, end11, start20, end20, start21, end21, Rtime_ch0, Rtime_ch2 = TimePeriodBorders (my_file)
    #print(my_file) #File is opened from the folder
    
    #Rin ch0
    currents_dev1, voltages_dev1 = IVpoints (my_file, 0, 1, start11, end11, Rtime_ch0, [], [])        
    Rin_ch0, R2_ch0 = Rin_R2_calculation (currents_dev1, voltages_dev1)
    
    Rin_ch0_list.append(Rin_ch0)
    R2_ch0_list.append(R2_ch0)
    number_ch0_list.append(int(len(voltages_dev1)))
    
    #Rin ch2
    currents_dev2, voltages_dev2 = IVpoints (my_file, 2, 3, start21, end21, Rtime_ch2, [], [])
    Rin_ch2, R2_ch2 = Rin_R2_calculation (currents_dev2, voltages_dev2)
    
    Rin_ch2_list.append(Rin_ch2)
    R2_ch2_list.append(R2_ch2)
    number_ch2_list.append(int(len(voltages_dev2)))
    
    

col1 = 'file_names'
col2 = 'Rin_ch0'
col3 = 'R2_ch0'
col4 = 'number_ch0'
col5 = 'Rin_ch2'
col6 = 'R2_ch2'
col7 = 'number_ch2'


my_data = pd.DataFrame({col1:file_names_list, 
                        col2:Rin_ch0_list, 
                        col3:R2_ch0_list,
                        col4:number_ch0_list, 
                        col5:Rin_ch2_list,
                        col6:R2_ch2_list,
                        col7:number_ch2_list})

    
my_data.to_excel(r'C:\Users\user\Documents\IMBIT\Master\Rin_R2_data.xlsx', sheet_name='All data', index=False)
print('Done!')

2023_08_21_0014.abf
2023_08_21_0015.abf
2023_08_21_0016.abf
2023_08_21_0017.abf
2023_08_21_0019.abf
2023_08_21_0020.abf
2023_08_21_0021.abf
2023_08_21_0022.abf
2023_08_21_0023.abf
2023_08_21_0024.abf
2023_08_21_0025.abf
2023_08_21_0026.abf
2023_08_21_0027.abf
2023_08_21_0028.abf
2023_08_21_0029.abf
Done!
